# Report

In [1]:
# This product uses the FRED® API but is not endorsed or certified by the Federal Reserve Bank of St. Louis
# import required python libraries

import pandas as pd
from functools import reduce
import numpy as np
import seaborn as sns
from datetime import *
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from fredapi import Fred
fred = Fred(api_key='03f93d3b6b484b4d658e5e7f044fd3fb')
# API key needs to be requested on the FRED data base

# disable warnings 
import warnings
warnings.filterwarnings('ignore')

### Extracting Data

In [2]:
# read in indicator data
GDP = pd.DataFrame(fred.get_series_latest_release('GDP')).reset_index()
unemployment_rate = pd.DataFrame(fred.get_series_latest_release('UNRATE')).reset_index()
mortgage_rate = pd.DataFrame(fred.get_series_latest_release('MORTGAGE30US')).reset_index()
CPI = pd.DataFrame(fred.get_series_latest_release('CPIAUCSL')).reset_index()
Tbill_year = pd.DataFrame(fred.get_series_latest_release('DTB1YR')).reset_index()
HPI = pd.DataFrame(fred.get_series_latest_release('USSTHPI')).reset_index()
ic = pd.DataFrame(fred.get_series_latest_release('ICSA')).reset_index()
disp_inc = pd.DataFrame(fred.get_series_latest_release('A229RX0')).reset_index()
consumer_con = pd.DataFrame(fred.get_series_latest_release('CSCICP03USM665S')).reset_index()
business_con = pd.DataFrame(fred.get_series_latest_release('BSCICP03USM665S')).reset_index()

macro_indicators = [GDP, unemployment_rate, mortgage_rate, CPI, Tbill_year, 
                    HPI, ic, disp_inc, consumer_con, business_con]

# read in target data
target = pd.read_excel('/Users/risshail/Downloads/Target_Variable_Risshail.xlsx',sheet_name='Sheet1')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/risshail/Downloads/Target_Variable_Risshail.xlsx'

In [ ]:
# index to time and standardize using datetime
def fix_db(db):
    db.columns = ("Time", "indicator")
    db["Time"] = pd.to_datetime(db["Time"],format='%Y-%m-%d')
    
for ind in macro_indicators:
    fix_db(ind)
    
# alter individual dataframes    
GDP["GDP_growth_rate"]=GDP["indicator"].pct_change()*100
GDP.drop(["indicator"], axis = 1, inplace = True)  

# merge dataframs across time
indicators_merged = reduce(lambda left,right: pd.merge(left,right,on=['Time'],
                                              how='outer'), macro_indicators);
indicators_merged.columns = ("YYYY-MM-DD", "GDP_growth_rate", "unemployment_rate", "mortgage_rate", 
                             "CPI", "treasury_bill_rate_yr", "HPI", "inital_claims(weekly)", 
                             "disposeable_income", "consumer_confidence", "business_confidence");
indicators_merged = indicators_merged[indicators_merged["YYYY-MM-DD"] >= datetime(2019, 1, 1)]
indicators_merged = indicators_merged.sort_values("YYYY-MM-DD", 
                                                  ascending = True).reset_index().drop("index", axis = 1)


# create dates column and merge
date_range = pd.DataFrame(pd.to_datetime(pd.date_range(start = indicators_merged["YYYY-MM-DD"].iloc[0], 
                                                       end = indicators_merged["YYYY-MM-DD"].iloc[-1], 
                                                       freq='D'),format='%Y-%m-%d'))
date_range.columns = ["YYYY-MM-DD"]
indicators = pd.merge(date_range, indicators_merged, on=["YYYY-MM-DD"], how='outer')

# organise target variable and index to time
target["YYYY-MM-DD"] = pd.to_datetime(pd.date_range(end = "2022-07-12", 
                                                    periods=len(target), freq='W'),format='%Y-%m-%d')
target.drop("Week#", axis = 1, inplace = True)
target.columns = ["Target", "YYYY-MM-DD"]
target = target[["YYYY-MM-DD", "Target"]]

In [ ]:
indicators.head()

In [ ]:
target.head()

In [ ]:
# find weekly averages for indicator data and merge the data with the target vairable and fill missing values
weekly = indicators.ffill().groupby([[i//7 for i in range(0, len(indicators))]], 
                                      axis = 0).mean()

weekly_date_range = pd.period_range(start = indicators["YYYY-MM-DD"].iloc[0], 
                                    end = indicators["YYYY-MM-DD"].iloc[-1], freq = 'W-SAT')
weekly_date_range = weekly_date_range.map(str)
weekly_date_range = weekly_date_range.str.split('/').str[0]
weekly_date_range = pd.Series(weekly_date_range)

weekly["YYYY-MM-DD"] = pd.to_datetime(weekly_date_range)

# merge weekly values with target variable
df = pd.merge(target, weekly,  on = ["YYYY-MM-DD"], how = "outer")
df = df.sort_values("YYYY-MM-DD", ascending = True).reset_index().drop("index", axis = 1)
df = df[df["Target"].isnull() == False].reset_index().drop(["index"], axis = 1)
df = df.set_index("YYYY-MM-DD")

In [ ]:
df.head()

### Deaseasonalizing Data

In [ ]:
# identify data seasonality
plt.figure(figsize=(12,6),dpi=150)
sns.scatterplot(x='YYYY-MM-DD',y='Target',data=df,color='black')
plt.axvline(x = pd.to_datetime("2019-07-14"), color = "red")
plt.axvline(x = pd.to_datetime("2020-07-05"), color = "red")
plt.axvline(x = pd.to_datetime("2021-07-04"), color = "red")
plt.axvline(x = pd.to_datetime("2022-07-03"), color = "red")
plt.suptitle("Target Values Plot Against Time")
plt.title("red stripes represent first week of september and lowest value each year");

# target variable is at minimum in the first week of September each year

In [ ]:
# decomposing data to identify trend after deseasonalizing
plt.rcParams['figure.figsize'] = (17,8)
decomposition = sm.tsa.seasonal_decompose(df['Target'], model='additive', period = 52)
fig = decomposition.plot()
plt.show()

In [ ]:
# comparing mulitplicative and additive deseasonalisation
fig,axes = plt.subplots(nrows=2,ncols=1)

result_add = sm.tsa.seasonal_decompose(df['Target'], model='additive', period=52)
deseasonalized_add = df['Target'].values - result_add.seasonal
axes[0].plot(deseasonalized_add)
axes[0].set_title('Target Deseasonalized Additive', fontsize=16)

result_mul = sm.tsa.seasonal_decompose(df['Target'], model='multiplicative', period=52)
deseasonalized_mul = df['Target'].values/result_mul.seasonal
axes[1].plot(deseasonalized_mul)
axes[1].set_title('Target Deseasonalized Multiplicative', fontsize=16)

plt.plot();

In [ ]:
# using multiplicative deseasonalization because of lower volatility
desea_df = df
desea_df["desea_target"] = df['Target'].values / result_mul.seasonal
desea_df = desea_df.drop(["Target"], axis = 1)
cols = desea_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
desea_df = desea_df[cols]

In [ ]:
desea_df.head()

### Picking Indicators

In [ ]:
# plotting all variables against time to understand trend
fig, axes = plt.subplots(nrows=5, ncols=2, dpi=120, figsize=(10,10))
for i, ax in enumerate(axes.flatten()):
    data = desea_df[desea_df.columns[i]]
    ax.plot(data, color='red', linewidth=1)
    # Decorations
    ax.set_title(desea_df.columns[i])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

In [ ]:
# deseasonalised target against indicators
sns.pairplot(data = desea_df, 
             y_vars = "desea_target");

In [ ]:
# correlation of indicators with deseasonalised target
desea_df.corr()

### Vector Autoregression Test

In [ ]:
#import modules
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

#### Seasonal Data

In [ ]:
# creating testing and training data sets
# data will be tested on the last 12 weeks of available data
vardf = df[["Target", "mortgage_rate"]]
train_df = vardf[:-12]
test_df = vardf[-12:]

In [ ]:
# check stationarity of data
ad_fuller_result_1 = adfuller(df['Target'].diff()[1:])

print('deseasonalised_target')
print(f'ADF Statistic: {ad_fuller_result_1[0]}')
print(f'p-value: {ad_fuller_result_1[1]}')

print('\n---------------------\n')

ad_fuller_result_2 = adfuller(df['mortgage_rate'].diff()[2:])

print('mortgage_rate')
print(f'ADF Statistic: {ad_fuller_result_2[0]}')
print(f'p-value: {ad_fuller_result_2[1]}')

In [ ]:
test_var_model = VAR(train_df.diff()[2:].dropna())

In [ ]:
test_sorted_order = test_var_model.select_order(maxlags=10)
print(test_sorted_order.summary())

In [ ]:
test_VAR_model = VARMAX(train_df, order=(4,0), enforce_stationarity = True)
test_fitted_model = test_VAR_model.fit(disp=False)

In [ ]:
# Forecast target and mortgage rate for the next 12 weeks
n_forecast = 12
test_predict = test_fitted_model.get_prediction(start=len(train_df),end=len(train_df) + n_forecast-1)
test_predictions = test_predict.predicted_mean
test_predictions.rename_axis('YYYY-MM-DD', inplace = True)
test_predictions["Target"] = test_predictions["Target"].abs()
test_predictions.head()

In [ ]:
# plot target predictions against actual target values
plt.figure(figsize=(12,6),dpi=150)
plt.ylim(0, 180)
sns.scatterplot(x='YYYY-MM-DD',y='Target',data=test_df,color='black')
sns.scatterplot(x='YYYY-MM-DD',y='Target',data=test_predictions,color='red')
plt.title("predicted target values(red) vs actual target values(black)");

In [ ]:
test_vs_pred=pd.concat([test_df,test_predictions],axis=1)

In [ ]:
test_vs_pred.plot(figsize=(12,5))

In [ ]:
test_RMSE = np.sqrt(mean_squared_error(test_df["Target"], test_predictions["Target"]))
print("RMSE:", test_RMSE)

#### Deseasonalised Data

In [ ]:
# create dataframe with target and indicator
vardf = desea_df[["desea_target", "mortgage_rate"]]
print(vardf.shape)

In [ ]:
# check stationarity of data
ad_fuller_result_1 = adfuller(desea_df['desea_target'].diff()[2:])

print('deseasonalised_target')
print(f'ADF Statistic: {ad_fuller_result_1[0]}')
print(f'p-value: {ad_fuller_result_1[1]}')

print('\n---------------------\n')

ad_fuller_result_2 = adfuller(desea_df['mortgage_rate'].diff()[2:])

print('mortgage_rate')
print(f'ADF Statistic: {ad_fuller_result_2[0]}')
print(f'p-value: {ad_fuller_result_2[1]}')

In [ ]:
# split data into training and testing data
train_df = vardf[:-12]
test_df = vardf[-12:]

In [ ]:
print(train_df.shape)

In [ ]:
test_var_model = VAR(train_df.diff()[2:].dropna())

In [ ]:
test_sorted_order = test_var_model.select_order(maxlags=10)
print(test_sorted_order.summary())

In [ ]:
test_VAR_model = VARMAX(train_df, order=(4,0), enforce_stationarity = True)
test_fitted_model = test_VAR_model.fit(disp=False)

In [ ]:
# Forecast target and mortgage rate for the next 12 weeks
n_forecast = 12
test_predict = test_fitted_model.get_prediction(start=len(train_df),end=len(train_df) + n_forecast-1)
test_predictions = test_predict.predicted_mean
test_predictions.rename_axis('YYYY-MM-DD', inplace = True)
test_predictions["desea_target"] = test_predictions["desea_target"].abs()
test_predictions.head()

In [ ]:
# plot target predictions against actual target values
plt.figure(figsize=(12,6),dpi=150)
plt.ylim(0, 200)
sns.scatterplot(x='YYYY-MM-DD',y='desea_target',data=test_df,color='black')
sns.scatterplot(x='YYYY-MM-DD',y='desea_target',data=test_predictions,color='red')
plt.title("predicted target values(red) vs actual target values(black)");

In [ ]:
test_vs_pred=pd.concat([test_df,test_predictions],axis=1)

In [ ]:
test_vs_pred.plot(figsize=(12,5));

In [ ]:
test_RMSE = np.sqrt(mean_squared_error(test_df["desea_target"], test_predictions["desea_target"]))
print("RMSE:", test_RMSE)

In [ ]:
# Even though the RMSE is lower with seasonal data, the model isn't picking up the trend 
# therefore deseasonalised data is used for final predictions

### Vector Autoregression Predictions

In [ ]:
# use the entire set of deseasonalised data
pred_var_model = VAR(vardf.diff()[2:])

In [ ]:
pred_sorted_order = pred_var_model.select_order(maxlags=10)
print(pred_sorted_order.summary())

In [ ]:
pred_VAR_model = VARMAX(vardf, order=(8,0),enforce_stationarity= True)
pred_fitted_model = pred_VAR_model.fit(disp=False)

# Predict target values for next 12 weeks
n_forecast = 12
pred_predict = pred_fitted_model.get_prediction(start=len(vardf),end=len(vardf) + n_forecast-1)
pred_predictions = pred_predict.predicted_mean
pred_predictions.rename_axis('YYYY-MM-DD', inplace = True)
pred_predictions["desea_target"] = pred_predictions["desea_target"].abs()

In [ ]:
pred_predictions.head()

In [ ]:
# plot deseasonalised predicted target values against actual target values
plt.figure(figsize=(12,6),dpi=150)
sns.scatterplot(x='YYYY-MM-DD',y='desea_target',data=vardf,color='black')
sns.scatterplot(x='YYYY-MM-DD',y='desea_target',data=pred_predictions,color='red')
plt.title("Predicted deseasonalised target values for the next 12 weeks");

In [ ]:
predicted_df = vardf.append(pred_predictions)
print(predicted_df.shape)

### Seasonalizing the data

In [ ]:
# extending seasonal data to match the length of the overall data
seasonal = pd.DataFrame(result_mul.seasonal)
seasonal = seasonal[:52].reset_index(drop = True)
seasonal = seasonal.iloc[np.tile(np.arange(len(seasonal)), 4)].reset_index()
seasonal = seasonal.drop("index", axis = 1)
seasonal = seasonal.iloc[:len(predicted_df)]

In [ ]:
# multiply seasonal values with deseasonalised predictions to get seasonalised predictions
predicted_df = predicted_df.reset_index()
seasonal_predictions = pd.merge(predicted_df, seasonal, left_index = True, right_index = True)
seasonal_predictions.set_index("YYYY-MM-DD", inplace = True)
seasonal_predictions["Target"] = seasonal_predictions["desea_target"] * seasonal_predictions["seasonal"]
seasonal_predictions.tail(12)

In [ ]:
plt.figure(figsize=(12,6),dpi=150)
sns.scatterplot(x='YYYY-MM-DD',y=seasonal_predictions["Target"].iloc[:-12],data=seasonal_predictions,color='black');
sns.scatterplot(x='YYYY-MM-DD',y=seasonal_predictions["Target"].iloc[-12:],data=seasonal_predictions,color='red')
plt.title("Predicted seasonalised target values for the next 12 weeks");